# Chapter 3: LangChain Fundamentals

## The Engine Room of Agentic AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPOSITORY/blob/main/notebooks/03-LangChain-Fundamentals.ipynb)

Welcome to the engine room! LangChain is arguably the most popular and comprehensive framework for building applications with Large Language Models. In this chapter, we'll dissect its fundamental components: Chains, Tools, Agents, and Memory.

## ⛓️ The Core Idea: Chains (using LCEL)

The heart of LangChain is the **chain**. A chain allows you to combine different components (like LLMs, prompts, and tools) in a sequence. The modern way to build chains is with the **LangChain Expression Language (LCEL)**, which uses the pipe operator `|` to connect components.

Think of it as a production line: an input goes in one end, passes through several stations, and a final output comes out the other end.

In [ ]:
# Step 1: Install and Setup
!pip install langchain langchain_google_genai python-dotenv

import os
from dotenv import load_dotenv
if not load_dotenv():
    try:
        from google.colab import userdata
        os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    except ImportError:
        print("Could not load API keys.")

In [ ]:
# Step 2: Build a simple LCEL Chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. Prompt Template
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

# 2. LLM
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# 3. Output Parser
output_parser = StrOutputParser()

# 4. Create the chain using LCEL
joke_chain = prompt | llm | output_parser

# 5. Run the chain
topic = "ice cream"
print(f"Joke about {topic}:\n")
print(joke_chain.invoke({"topic": topic}))

## 🛠️ Giving Your Chain Superpowers: Tools

What if your LLM needs to know something it wasn't trained on, like today's date or the result of a complex calculation? That's where **Tools** come in. A tool is a function that an agent or chain can use to interact with the outside world.

In [ ]:
# Step 3: Define a custom tool
from langchain.tools import tool
import datetime

@tool
def get_current_date(text: str) -> str:
    """Returns the current date, useful for any questions about today's date."""
    return str(datetime.date.today())

# Let's test the tool
print(get_current_date.name)
print(get_current_date.description)
print(get_current_date.run(""))

## 🧠 The Brain of the Operation: Agents Revisited

An **Agent** is a special type of chain that has an LLM at its core which can decide which, if any, tools to use. It uses a framework like **ReAct (Reasoning and Acting)** to loop through a cycle of Thought -> Action -> Observation until it has an answer.

In [ ]:
# Step 4: Create an Agent with our custom tool
from langchain.agents import initialize_agent, AgentType

tools = [get_current_date]
agent = initialize_agent(
    tools,
    llm, # Our LLM from before
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Test the agent
agent.run("What date is it today?")

## 🧠 Don't Forget! Adding Memory

By default, chains and agents are stateless. They treat each new query as a completely separate event. To have a real conversation, we need to add **Memory**. Memory allows a chain or agent to remember previous interactions.

In [ ]:
# Step 5: Add Memory to a Chain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Initialize memory
memory = ConversationBufferMemory()

# Create a ConversationChain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Start the conversation
print("--- First Interaction ---")
conversation.predict(input="Hi, my name is Jules.")

print("\n--- Second Interaction ---")
conversation.predict(input="What is my name?")

# You can inspect the memory
print("\n--- Memory Contents ---")
print(memory.buffer)

## ✅ Putting It All Together

You've now learned about the four pillars of LangChain:

1.  **Chains (LCEL):** For composing sequences of operations.
2.  **Tools:** For giving your chains and agents access to the real world.
3.  **Agents:** For making decisions and using tools to accomplish goals.
4.  **Memory:** For remembering past interactions.

Mastering these concepts is the key to unlocking the full potential of LangChain.

In the next chapter, we'll shift our focus to a higher-level framework, **CrewAI**, which is specifically designed for orchestrating multi-agent collaboration. You'll see how it builds on the foundational ideas we've covered here.